In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import math

In [2]:
f = pd.read_csv("spambase.data", header = None)
train, test = train_test_split(f, train_size = 2/3)#Random splitting

In [3]:
def Zscore(x, dataset):#Re-used from last homework(s)
    for i in range(dataset.shape[1] - 1):
        col = dataset.iloc[:, i].to_numpy()
        temp = (col - np.mean(col))/ (np.std(col, ddof = 1))
        x = np.append(x, temp.reshape(-1, 1), axis = 1)
    return x

In [4]:
bias_train = np.ones(train.shape[0]).reshape(-1, 1)
bias_test = np.ones(test.shape[0]).reshape(-1, 1)
zscored_train = pd.DataFrame(Zscore(bias_train, train.iloc[:, :-1]))
zscored_test = pd.DataFrame(Zscore(bias_test, test.iloc[:, :-1]))
X = zscored_train
Y = test.iloc[:, -1].to_numpy()
X.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,1.0,-0.348066,-0.164411,-0.559200,-0.048325,-0.456769,-0.351222,-0.288121,-0.253302,-0.338807,...,-0.071767,-0.102237,-0.151829,-0.494795,-0.144416,1.094799,-0.294335,-0.122611,-0.141426,-0.240009
1,1.0,-0.348066,1.383830,-0.559200,-0.048325,-0.456769,-0.351222,-0.288121,-0.253302,-0.338807,...,-0.071767,-0.102237,0.243771,-0.098018,-0.144416,-0.188260,-0.294335,-0.122611,-0.087883,-0.176564
2,1.0,3.049347,-0.164411,1.593409,-0.048325,1.087371,3.486176,-0.288121,-0.253302,-0.338807,...,-0.071767,-0.102237,-0.151829,0.815985,-0.144416,-0.108206,-0.294335,-0.122611,-0.096687,-0.144842
3,1.0,-0.348066,-0.164411,0.486928,-0.048325,-0.456769,-0.351222,-0.288121,-0.253302,-0.338807,...,-0.071767,-0.102237,-0.151829,-0.229096,1.184919,-0.311083,-0.294335,-0.122611,-0.090337,-0.162969
4,1.0,0.064704,-0.064276,-0.297668,-0.048325,0.336948,0.617094,0.398944,0.057602,3.815415,...,-0.071767,-0.102237,-0.151829,-0.434570,-0.144416,1.132084,0.397652,-0.122611,0.000982,0.806835


In [ ]:
#Converting into binary dataset
#Takes a lot of time
for i in range(X.shape[1]):
    for j in range(len(X)):
        if X.iloc[j, i] >= X.iloc[:, i].mean():
            X.iloc[j, i] = 1
        else:
            X.iloc[j, i] = 0
X.head()

In [ ]:
#To find target entropy
target_zeroes, target_ones = X[X.shape[1] - 1].value_counts()
target_entropy = -(target_zeroes / X.shape[0]) * math.log(target_zeroes / X.shape[0], 2) -\
(target_ones / X.shape[0]) * math.log(target_ones / X.shape[0], 2) 
target_entropy

In [ ]:
def get_entropy(X):
    '''
    Returns entropies of each column in form of a dictionary, from a dataset X
    '''
    #First column, used this for testing as well
    #Weirdly doing this manually produces agreeable entropy, whereas in the loop in next cell this gives negative values
    #Might be because there's only one element in tmp0[0]
    tmp1 = X[X[0] == 1]
    tmp0 = X[X[0] == 0]
    zero1, one1 = tmp1[tmp1.shape[1] - 1].value_counts()
    E1 = -(zero1 / tmp1.shape[0] * math.log(zero1 / tmp1.shape[0], 2)) -\
            (one1 / tmp1.shape[0] * math.log(one1 / tmp1.shape[0], 2))
    entropy = {0: E1 * (tmp1.shape[0]/X.shape[0])}
    
    for i in range(1, X.shape[1]):
        tmp0 = X[X[i] == 0]
        tmp1 = X[X[i] == 1]
        #For value == 0
        try:
            zero0, one0 = tmp0[tmp0.shape[1] - 1].value_counts()
        except ValueError:
            pass
        try:
            E0 = -(zero0 / tmp0.shape[0] * math.log(zero0 / tmp0.shape[0], 2)) -\
            (one0 / tmp0.shape[0] * math.log(one0 / tmp0.shape[0], 2))
        except NameError:
            E0 = 0#If there's only one element, it gives me these errors
        #For value == 1
        try:
            zero1, one1 = tmp1[tmp1.shape[1] - 1].value_counts()
        except ValueError:
            pass
        try:
            E1 = -(zero1 / tmp1.shape[0] * math.log(zero1 / tmp1.shape[0], 2)) -\
            (one1 / tmp1.shape[0] * math.log(one1 / tmp1.shape[0], 2))
        except NameError:
            E1 = 0

        entropy[i] = E0 * (tmp0.shape[0]/X.shape[0]) + E1 * (tmp1.shape[0]/X.shape[0])
    #entropy#Testing
    info_gain = {}
    for k in entropy:
        info_gain[k] = target_entropy - entropy[k]
    
    return info_gain

In [ ]:
class tree:
    def __init__(self, col, x):
        self.col = col
        self.x = x
    def check(self, data):
        x = data[self.col]
        return x == self.x
    #Incomplete

def get_max_entropy(X):
    gain = get_entropy(X)
    max = 0
    for i in gain:
        if max < gain[i]:
            max = gain[i]
    return max, i

In [ ]:
max_entropy, col = get_max_entropy(X)#For dominant feature
max_entropy, col

Things left to do : Creating tree using the class tree and predicting values. But already past the dealine so I'll take reduction in marks and keep working on this(for myself).

In [ ]:
di = {col:max_entropy}#Starting point
Xnew = X

for i in range(X.shape[1]):
    try:
        Xnew = Xnew.drop(columns = col)
        max_new, col_new = get_max_entropy(Xnew)
    except KeyError:
        pass
    di[col_new] = max_new